# Constants and libraries

In [1]:
'''
In this section one defines environment variables. 
Because I used this notebook on number of machines, I implemented class especially for this. 
You may not needed in one and use just simple definitions.
'''

from system_variables import SystemVariables

# choose system according your current machine
# SYSTEM_NAME = "Windows"
# SYSTEM_NAME = "Colab"
# SYSTEM_NAME = "Kaggle"
SYSTEM_NAME = "Linux"

sv = SystemVariables(SYSTEM_NAME)
PROJECT_FOLDER = sv.get_project_folder()
SRC_FOLDER = sv.get_src_folder()
OUTPUT_FOLDER = sv.get_output_folder()
TUAB_DIRECTORY, TUAB_TRAIN, TUAB_EVAL = sv.get_TUAB_folders()
DEPR_ANON_DIRECTORY = sv.get_depr_anon_folder()
INHOUSE_DIRECTORY = sv.get_inhouse_folder()

print(SYSTEM_NAME)
print()

print(f"{PROJECT_FOLDER = }")
print(f"{SRC_FOLDER = }")
print(f"{OUTPUT_FOLDER = }")
print()

print(f"{TUAB_DIRECTORY = }")
print(f"{TUAB_TRAIN = }")
print(f"{TUAB_EVAL = }")
print()

print(f"{DEPR_ANON_DIRECTORY = }")
print()

print(f"{INHOUSE_DIRECTORY = }")
print()

Linux

PROJECT_FOLDER = ''
SRC_FOLDER = ''
OUTPUT_FOLDER = ''

TUAB_DIRECTORY = 'Data/TUAB/'
TUAB_TRAIN = 'Data/TUAB/train/normal/01_tcp_ar/'
TUAB_EVAL = 'Data/TUAB/eval/normal/01_tcp_ar/'

DEPR_ANON_DIRECTORY = 'Data/depression_anonymized/'

INHOUSE_DIRECTORY = 'Data/inhouse_dataset/EEG_baseline_with_markers_cleaned/preprocessed_data/EEG_baseline/'



In [2]:
import warnings
warnings.simplefilter("ignore")

import os
import sys
import pickle
import json
import random
from IPython.display import display, clear_output
from tqdm.auto import tqdm as tqdm_auto

In [3]:
sys.path.append(SRC_FOLDER)

%load_ext autoreload
%autoreload 1
%aimport utils

from utils import SEED
from utils.common import check_tags, remove_files
from utils.data_reading import DataReader
from utils.plotting import dataset_hists
from utils.data_preprocessing import preprocessDepressionAnonymizedData, preprocessInhouseDatasetData, preprocessTUABdata, getTUABdataset, save_preprocessed_data

# Data preview

In [16]:
def tree(data_dir, n_files=2):
    for dir, _, filenames in os.walk(data_dir):
        print(dir)
        for filename in filenames[:n_files]:
            print(os.path.join(dir, filename))

# tree(DEPR_ANON_DIRECTORY)

# tree(INHOUSE_DIRECTORY)


# tree(TUAB_DIRECTORY)
# raw = mne.io.read_raw_edf(TUAB_TRAIN + "/aaaaaaff_s002_t000.edf", preload=False)
# print(raw.ch_names, "\nDuration:", raw.times[-1]/60, "m")
# display(raw.info)
# raw.plot(n_channels=21, duration=100, scalings=100e-6, start=20);#.set_size_inches(5, 5, forward=True)

## Inhouse dataset

In [25]:
import pandas as pd
from tqdm import tqdm 

for dir in ["Health/", "MDD/"]:
    print(len(os.listdir(INHOUSE_DIRECTORY + dir)))
    n_secs = 0
    for file in tqdm(os.listdir(INHOUSE_DIRECTORY + dir)):
        n_secs += len(pd.read_csv(INHOUSE_DIRECTORY + dir + file)) / 500
    print(n_secs/len(os.listdir(INHOUSE_DIRECTORY + dir)), n_secs)

75


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:09<00:00,  7.96it/s]


96.56 7242.0
74


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:09<00:00,  8.12it/s]

95.1891891891892 7044.0


## Depression anonymized dataset

In [32]:
for dir in ["epoch_1/", "epoch_3/"]:
    print(len(os.listdir(DEPR_ANON_DIRECTORY + dir)))
    
    path_file = pd.read_csv(DEPR_ANON_DIRECTORY + dir + 'path_file.csv')
    print(path_file['target'].value_counts())

    n_secs = 0
    for file in tqdm(os.listdir(DEPR_ANON_DIRECTORY + dir)):
        if "path_file" in file or 'ipynb_checkpoints' in file:
            continue
        recording = pd.read_csv(DEPR_ANON_DIRECTORY + dir + file)
        n_secs += len(recording)/125
    print(n_secs/len(os.listdir(DEPR_ANON_DIRECTORY + dir)), n_secs)

91
target
0    45
1    44
Name: count, dtype: int64


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91/91 [00:02<00:00, 37.32it/s]


58.68131868131868 5340.0
91
target
0    45
1    44
Name: count, dtype: int64


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91/91 [00:03<00:00, 23.45it/s]

58.68131868131868 5340.0


# Data preprocessing and saving

### TUAB

In [ ]:
#preprocess
for n_secs in [1, 2, 4, 5, 10, 15, 30, 60]:
    picked_channels = ['EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF']
    target_freq = 128
    n_samples_per_chunk=target_freq*n_secs
    
    chunks_file_name = f"chunks_fz_cz_pz_{len(picked_channels)}x{n_samples_per_chunk}"
    targets_file_name = f"targets_fz_cz_pz_{len(picked_channels)}x{n_samples_per_chunk}"

    remove_files(TUAB_TRAIN, [chunks_file_name, targets_file_name])
    remove_files(TUAB_EVAL, [chunks_file_name, targets_file_name]) 
    
    preprocessTUABdata(
        picked_channels, 
        chunks_file_name, 
        targets_file_name, 
        TUAB_TRAIN, 
        TUAB_EVAL, 
    
        target_freq=target_freq,
        n_samples_per_chunk=n_samples_per_chunk,
        
        force_recompute=True,
        is_save=True,
        n_files=None,
        n_chunks_max=None,
    )
    
    dataset = getTUABdataset(
        TUAB_TRAIN, 
        TUAB_EVAL, 
        chunks_file_name, 
        targets_file_name,
        
        n_channels=len(picked_channels),
        n_samples_per_chunk=n_samples_per_chunk,
        test_size=0.5,
        SEED=SEED,
    )
    
    save_preprocessed_data(
        dataset, 
        os.path.join(TUAB_DIRECTORY, f"dataset_{target_freq}_{n_samples_per_chunk/target_freq:.1f}.pkl")
    )

4 files was removed from Data/TUAB/train/normal/01_tcp_ar/
6 files was removed from Data/TUAB/eval/normal/01_tcp_ar/


  0%|▏                                                                                                                      | 2/1383 [00:07<1:29:47,  3.90s/it]

File aaaaanle_s001_t000.edf was passed, passed files: 1, read files: 2


  0%|▌                                                                                                                      | 6/1383 [00:19<1:15:30,  3.29s/it]

File aaaaapik_s001_t001.edf was passed, passed files: 2, read files: 5


...

File aaaaaosw_s001_t000.edf was passed, passed files: 12, read files: 59


 54%|████████████████████████████████████████████████████████████████▌                                                       | 341/634 [02:51<03:29,  1.40it/s]

File aaaaanbq_s001_t000.edf was passed, passed files: 13, read files: 67
File aaaaappb_s001_t000.edf was passed, passed files: 14, read files: 67
File aaaaapqv_s001_t000.edf was passed, passed files: 15, read files: 67


 59%|██████████████████████████████████████████████████████████████████████▉                                                 | 375/634 [03:03<02:10,  1.99it/s]

File aaaaapof_s001_t000.edf was passed, passed files: 16, read files: 72


### Depression anonymized

In [ ]:
# epoch_folders = ["epoch_1/", "epoch_3/"]
# picked_channels = ["fz", "cz", "pz"]
# source_freq = 125
# target_freq = 128
# n_samples_per_chunk = target_freq*5

# chunks_list = preprocessDepressionAnonymizedData(
#     DEPR_ANON_DIRECTORY, 
#     epoch_folders, 
#     picked_channels,
#     target_freq = target_freq,
#     n_samples_per_chunk = n_samples_per_chunk
# )
# save_preprocessed_data(chunks_list, DEPR_ANON_DIRECTORY + f"dataset_{target_freq}_{n_samples_per_chunk/target_freq:.1f}.pkl")

### Inhouse dataset

In [ ]:
# for n_secs in [1, 2, 4, 5, 10, 15, 30, 60]:
#     data_folders = ["MDD", "Health"]
#     picked_channels = ["Fz", "Cz", "Pz"]
#     source_freq = 500
#     target_freq = 128
#     n_samples_per_chunk = target_freq*n_secs
    
#     chunks_list = preprocessInhouseDatasetData(
#         INHOUSE_DIRECTORY, 
#         data_folders, 
#         picked_channels,
#         source_freq = source_freq,
#         target_freq = target_freq,
#         n_samples_per_chunk = n_samples_per_chunk
#     )
#     save_preprocessed_data(chunks_list, INHOUSE_DIRECTORY + f"dataset_{target_freq}_{n_samples_per_chunk/target_freq:.1f}.pkl")

# Data load


In [ ]:
# file = DEPR_ANON_DIRECTORY + "dataset_128_10.0.pkl"
# train_set, val_set, test_set = DataReader(file, dataset_type="depression_anonymized").split()

file = TUAB_DIRECTORY + "dataset_128_1.0.pkl"
train_set, val_set, test_set = DataReader(file, dataset_type="TUAB").split()

# file = INHOUSE_DIRECTORY + "dataset_128_60.0.pkl"
# train_set, val_set, test_set = DataReader(file, dataset_type="inhouse_dataset").split()

chunks_train, chunks_val, chunks_test = train_set["chunk"], val_set["chunk"], test_set["chunk"]
targets_train, targets_val, targets_test = train_set["target"], val_set["target"], test_set["target"]

# Data exploration

In [ ]:
# dataset_hists(train_set, val_set, test_set, chunk_bins=50, chunk_range=(-5, 5))
dataset_hists(train_set, val_set, test_set, chunk_bins=50, chunk_range=(-5, 5), target_bins=10, target_range=(0, 100))